In [1]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

##### Funcion para descargar los datos y EDA del cluster

In [2]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL_nits = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', CONCAT(CONCAT(VTANIT, '-'), VTASUC) AS 'Nitcliente-sucursal', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1), CONCAT(CONCAT(VTANIT, '-'), VTASUC)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL_nits = pd.concat([df_SQL_nits, df_SQL_int], ignore_index= True)

    df_SQL = df_SQL_nits.groupby("Fecha").sum().reset_index()
    df_SQL_nits = df_SQL_nits.groupby("Nitcliente-sucursal").sum().reset_index()

    return df_SQL, df_SQL_nits

In [3]:
def lineplot(bd):
    x = bd["Fecha"]
    x_n = np.arange(0, len(bd))
    y = bd["Ventas"]   
    coeficientes = np.polyfit(x_n, y, 1)
    poli = np.poly1d(coeficientes)

    trace1 = go.Scatter(x=x, y=y, mode='lines+markers', name='Ventas')
    trace2 = go.Scatter(x=x, y=poli(x_n), mode='lines', name='Línea de Tendencia')

    layout = go.Layout(
            title='Ventas por mes',
            xaxis=dict(title='Fecha'),
            yaxis=dict(title='Ventas'),
            legend=dict(x=1, y=1)
    )

    fig = go.Figure()
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(layout)
    fig.show()

def EDA_cluster(bd, bd_nits):
    #Variables
    bd = bd.reset_index()
    bd_nits = bd_nits
    primer_fecha = datetime.utcfromtimestamp(bd.iloc[0, 1].timestamp())
    ultima_fecha = datetime.utcfromtimestamp(bd.iloc[-1, 1].timestamp())
    describe_bd = bd.describe().applymap("{:,.0f}".format)
    describe_bd_nits = bd_nits.describe().applymap("{:,.0f}".format)
    
    print(f"Esta base de datos tiene ventas de {len(bd)} meses,")
    print(f"empezando desde el {primer_fecha.strftime('%d-%m-%Y')}")
    print(f"y terminando el {ultima_fecha.strftime('%d-%m-%Y')}")
    print("La composicion estadistica de la base de datos es la siguiente:")
    print(describe_bd["Ventas"][1:])
    print(" ")

    print(f"Por otro lado, esta compuesto por ventas de {len(bd_nits)} clientes")
    print("Y asi se comporta estadisticamente asi:")
    print(describe_bd_nits["Ventas"][1:])
    print(" ")


    lineplot(bd)

##### Carga de los clusters

In [51]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

#El mejor modelo es 10-Institucional-A
filtro_distrito = 90
filtro_tipozona = "Artes graficas"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                            (df_clusters["Codigo distrito"] == filtro_distrito) &
                            (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                            (df_clusters["Letra cluster"] == filtro_cluster)
                            ]

##### EDA del cluster elegido

In [52]:
df_clusters_EDA = df_clusters.groupby(["Codigo distrito", "Nombre tipo zona", "Letra cluster"]).agg({"Nit cliente-sucursal": np.size}).reset_index()

In [53]:
df_clusters_EDA

,Codigo distrito,Nombre tipo zona,Letra cluster,Nit cliente-sucursal
0,10,Artes graficas,A,61
1,10,Artes graficas,B,154
2,10,Artes graficas,C,135
3,10,Artes graficas,D,88
4,10,Artes graficas,E,85
...,...,...,...,...
330,90,Mayorista,A,14
331,90,Mayorista,B,23
332,90,Mayorista,C,32
333,90,Mayorista,D,31


In [54]:
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [55]:
ventas_cluster, ventas_nits = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [56]:
EDA_cluster(ventas_cluster, ventas_nits)

Esta base de datos tiene ventas de 63 meses,
empezando desde el 01-01-2018
y terminando el 01-03-2023
La composicion estadistica de la base de datos es la siguiente:
mean     94,682,981
std      45,395,182
min      35,286,793
25%      61,509,892
50%      85,204,355
75%     124,944,044
max     246,082,845
Name: Ventas, dtype: object
 
Por otro lado, esta compuesto por ventas de 14 clientes
Y asi se comporta estadisticamente asi:
mean      426,073,413
std       544,516,272
min         1,844,381
25%       120,978,890
50%       271,132,906
75%       514,459,546
max     2,181,572,306
Name: Ventas, dtype: object
 


##### Configuracion y prediccion

In [57]:
s = setup(
            ventas_cluster, #df
            target= "Ventas",
            ignore_features= ["Fecha"],
            session_id = 42, #id para mantener replicabilidad
            transform_target= None, #transformador del target, “box-cox”, “log”, “sqrt”, “exp”, “cos”
            coverage= 0.9, #intervalos
            verbose= False, #para no imprimir información irrelevante
            profile= True, #probar EDA interactivo
            fh = 6
            )

In [66]:
# remove_metric('RMSE')
# remove_metric('RMSSE')
# remove_metric('SMAPE')
# remove_metric('MAE')
top_3 = compare_models(
                        n_select= 3,
                        sort= "MAPE"
                        )
metricas_completas = pull()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:06:11
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [67]:
# Establecer el formato de visualización de números en notación decimal
pd.options.display.float_format = '{:,.3f}'.format
# metricas_completas
# metricas_completas[:3]

In [68]:
plot_model(top_3, plot = 'forecast')

In [69]:
plot_model(top_3, plot = 'diagnostics')

In [70]:
plot_model(top_3, plot = 'insample')

In [71]:
plot_model(top_3, plot = 'forecast', data_kwargs = {'fh': 9})

### Prediction

In [72]:
# predicciones
predicciones_modelo1 = predict_model(top_3[0], fh=  9)
predicciones_modelo2 = predict_model(top_3[1], fh=  9)
predicciones_modelo3 = predict_model(top_3[2], fh=  9)
predicciones = pd.DataFrame({metricas_completas.index[0]: np.squeeze(predicciones_modelo1.values),
                            metricas_completas.index[1]: np.squeeze(predicciones_modelo2.values),
                            metricas_completas.index[2]: np.squeeze(predicciones_modelo3.values)
                            },
                            index= predicciones_modelo1.index)

In [73]:
pd.options.display.float_format = '{:,.0f}'.format
predicciones

,et_cds_dt,xgboost_cds_dt,dt_cds_dt
57,"68,414,507","64,437,913","63,888,127"
58,"82,624,110","174,724,770","175,094,403"
59,"102,064,628","149,347,656","149,358,216"
60,"91,759,382","148,670,482","149,144,218"
61,"120,350,082","91,452,733","121,540,548"
62,"130,734,259","134,148,137","148,374,555"
63,"114,813,536","118,143,543","148,682,479"
64,"100,183,543","78,436,880","88,326,344"
65,"132,584,158","136,683,998","138,421,974"
